In [5]:
from sklearn.model_selection import ParameterGrid
from Trajlib2.SegmentationAlgorithms.GRASP_UTS import GRASPUTS#,GRASPUTSEvaluate
from sklearn.model_selection import KFold
import pandas as pd
import time,random

In [6]:
random.seed(2)

In [7]:
#Load Fishing Vessels
fvdfs = []
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d1.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d2.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d3.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d4.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d5.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d6.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d7.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d8.txt',sep=';'))
fvdfs.append(pd.read_csv('executable/databases/Fishing Vessels/fv_d9.txt',sep=';'))
real_fvdfs = []
for df in fvdfs:
    df.set_index(keys=['tid'],drop=False,inplace=True)
    tids = df['tid'].unique().tolist()
    for tid in tids:
        real_fvdfs.append(df.loc[df.tid ==tid])

In [8]:
#Split the data into 10 folds
splitter = KFold(n_splits=10,shuffle=True,random_state=2)
folds = splitter.split(real_fvdfs)
(train,test) = folds
print(train)
print(test)

ValueError: too many values to unpack (expected 2)

In [9]:
#Build Fishing Vessel Grid
alpha = [0.3]
partitioning_factor =[0]
max_iterations=[30]
min_times = [6,12,18,24]
jcs = [0.3,0.4]
feature_names = ['direction_inference','speed_inference_m_s']
grid = {'alpha':alpha, 'partitioning_factor': partitioning_factor, 'max_iterations':max_iterations,'min_time':min_times,'jcs':jcs}
parm_grid = ParameterGrid(grid)

In [10]:
def init_algo(p):
    grasputs = GRASPUTS(feature_names=feature_names,
                        lat_col='latitude', lon_col='longitude',
                        time_format='%Y-%m-%d %H:%M:%S',
                        min_time=p['min_time'],
                        join_coefficient=p['jcs'],
                        alpha=p['alpha'],
                        partitioning_factor=p['partitioning_factor'])
def execute(grasputs,dfs):
        df_out = pd.DataFrame()

In [11]:
df_out = []
for p in parm_grid:
    count=0
    grasputs = init_algo(p)
    print(p)
    for df in real_fvdfs:
        count+=1
        start_time = time.time()
        print("Trajectory", count)
        segs, cost = grasputs.segment(df)
        ground_truth = GRASPUTSEvaluate.get_ground_truth(df,label='label')
        prediction = GRASPUTSEvaluate.get_predicted(segs)
        print("Purity",GRASPUTSEvaluate.purity(ground_truth,prediction)[1])
        print("Coverage",GRASPUTSEvaluate.coverage(ground_truth,prediction)[1])
        print("Harmonic Mean",GRASPUTSEvaluate.harmonic_mean(ground_truth,prediction))
        print("--- %s seconds ---" % (time.time() - start_time))

TypeError: 'module' object is not callable